# **IndoSum Dataset EDA**
**Potato Tech** team build an app that named **Frasa** for capstone project. So we will build machine learning model for Frasa to predict a summary from a sentences.

In this lab, we will do exploratory data analysis and preprocess the dataset.
The dataset is consists of news articles and manually constructed summaries [IndoSum dataset](https://drive.google.com/file/d/1OgYbPfXFAv3TbwP1Qcwt_CC9cVWSJaco/view) from [Kurniawan, Kemal & Louvan, Samuel (2018)](https://arxiv.org/abs/1810.05334).

Let's start!

## Preparation

In [ ]:
!pip install Sastrawi

     |████████████████████████████████| 215kB 15.0MB/s 


In [ ]:
# Import library
import pandas as pd
import json

from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from nltk.tokenize.treebank import TreebankWordDetokenizer as untoken


## Load data

In [ ]:
# Download the data from drive
!gdown --id 1OgYbPfXFAv3TbwP1Qcwt_CC9cVWSJaco

Downloading...
From: https://drive.google.com/uc?id=1OgYbPfXFAv3TbwP1Qcwt_CC9cVWSJaco
To: /content/indosum.tar.gz
96.0MB [00:00, 186MB/s] 


In [ ]:
# Unzip

!tar -xvf /content/indosum.tar.gz

indosum/
indosum/test.02.jsonl
indosum/README.txt
indosum/train.05.jsonl
indosum/train.04.jsonl
indosum/test.04.jsonl
indosum/train.01.jsonl
indosum/test.05.jsonl
indosum/test.03.jsonl
indosum/test.01.jsonl
indosum/dev.04.jsonl
indosum/dev.01.jsonl
indosum/dev.02.jsonl
indosum/train.02.jsonl
indosum/dev.03.jsonl
indosum/dev.05.jsonl
indosum/CHANGELOG.txt
indosum/train.03.jsonl


In [ ]:
# We will use the first-fold data (train, dev, test)

url = "/content/indosum/train.01.jsonl"
with open(url, 'r') as f:
    data = f.readlines()
    train = list(map(json.loads, data)) 

train = pd.DataFrame(train)


## Exploratoratory

In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14262 entries, 0 to 14261
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   category     14262 non-null  object
 1   gold_labels  14262 non-null  object
 2   id           14262 non-null  object
 3   paragraphs   14262 non-null  object
 4   source       14262 non-null  object
 5   source_url   14262 non-null  object
 6   summary      14262 non-null  object
dtypes: object(7)
memory usage: 780.1+ KB


Now we know the dataset consist of 7 columns and probably we will just take some of the column that related to this summarization work.

To make we sure what columns should we choose, let's take a look at some rows.

In [ ]:
train.head()

,category,gold_labels,id,paragraphs,source,source_url,summary
0,tajuk utama,"[[False, True], [True, True], [False, False, F...",1501893029-lula-kamal-dokter-ryan-thamrin-saki...,"[[[Jakarta, ,, CNN, Indonesia, -, -, Dokter, R...",cnn indonesia,https://www.cnnindonesia.com/hiburan/201708041...,"[[Dokter, Lula, Kamal, yang, merupakan, selebr..."
1,teknologi,"[[False, False, False, False], [False, True, T...",1509072914-dua-smartphone-zenfone-baru-tawarka...,"[[[Selfie, ialah, salah, satu, tema, terpanas,...",dailysocial.id,https://dailysocial.id/post/dua-smartphone-zen...,"[[Asus, memperkenalkan, , ZenFone, generasi, ..."
2,hiburan,"[[True], [True], [False, False], [False], [Fal...",1510613677-songsong-visit-2020-bengkulu-perkua...,"[[[Jakarta, ,, CNN, Indonesia, -, -, Dinas, Pa...",cnn indonesia,https://www.cnnindonesia.com/gaya-hidup/201711...,"[[Dinas, Pariwisata, Provinsi, Bengkulu, kemba..."
3,tajuk utama,"[[True, True], [False, False, False], [True], ...",1502706803-icw-ada-kejanggalan-atas-tewasnya-s...,"[[[Merdeka.com, -, Indonesia, Corruption, Watc...",merdeka,https://www.merdeka.com/peristiwa/icw-merasa-a...,"[[Indonesia, Corruption, Watch, (, ICW, ), mem..."
4,tajuk utama,"[[False, True], [True, True, True], [False], [...",1503039338-pembagian-sepeda-usai-upacara-penur...,"[[[Merdeka.com, -, Presiden, Joko, Widodo, (, ...",merdeka,https://www.merdeka.com/peristiwa/usai-upacara...,"[[Jokowi, memimpin, upacara, penurunan, bender..."


Wow... that's a unique dataset we have here!

1. The news content and summary are raw text but listed (tokenized).
2. Punctuation has not been removed.
3. There are uppercase letter everywhere.
3. Column gold_labels is a boolean list of main idea sentences in paragraphs.

From this result, the columns will be used are **gold_labels, paragraphs**, and **summary**.

In [ ]:
df_train = train[['gold_labels', 'paragraphs', 'summary']].copy()  

df_train.info()
df_train.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14262 entries, 0 to 14261
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   gold_labels  14262 non-null  object
 1   paragraphs   14262 non-null  object
 2   summary      14262 non-null  object
dtypes: object(3)
memory usage: 334.4+ KB


,gold_labels,paragraphs,summary
0,"[[False, True], [True, True], [False, False, F...","[[[Jakarta, ,, CNN, Indonesia, -, -, Dokter, R...","[[Dokter, Lula, Kamal, yang, merupakan, selebr..."
1,"[[False, False, False, False], [False, True, T...","[[[Selfie, ialah, salah, satu, tema, terpanas,...","[[Asus, memperkenalkan, , ZenFone, generasi, ..."
2,"[[True], [True], [False, False], [False], [Fal...","[[[Jakarta, ,, CNN, Indonesia, -, -, Dinas, Pa...","[[Dinas, Pariwisata, Provinsi, Bengkulu, kemba..."
3,"[[True, True], [False, False, False], [True], ...","[[[Merdeka.com, -, Indonesia, Corruption, Watc...","[[Indonesia, Corruption, Watch, (, ICW, ), mem..."
4,"[[False, True], [True, True, True], [False], [...","[[[Merdeka.com, -, Presiden, Joko, Widodo, (, ...","[[Jokowi, memimpin, upacara, penurunan, bender..."


Let's check if we had empty value.

In [ ]:
df_train.isnull().any()

gold_labels    False
paragraphs     False
summary        False
dtype: bool

Great!

No empty value here from **14262 rows** data.

That's mean we can start preprocessing the dataset.
For text-type dataset, we need to do some steps:
1. Cleaning punctuation
2. Stemming
3. Remove stopwords
4. Lowercasing letter

Just try to create simple approach (not model, yet)

In [ ]:
import bs4
import urllib.request as url
import re
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk import sent_tokenize
from nltk.corpus import stopwords
from nltk import word_tokenize
stop_word = stopwords.words('english')
import string

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
url_name = input("Enter url of the text you want to summerize:")

Enter url of the text you want to summerize:https://en.wikipedia.org/wiki/Frances_Gertrude_McGill


In [ ]:
# web = url.urlopen(url_name)
# page = bs4.BeautifulSoup(web,'html.parser')
# elements = page.find_all('p')
# article = ''
# for i in elements:
#     article+= (i.text)
article = ""
with open('readme.txt') as f:
    lines = f.readlines()
    print(lines)
for i in lines:
    article+= i
article

['Suara.com - Moda transportasi Kereta Api Bandara International Soekarno - Hatta di Cengkareng akan dioperasikan mulai awal Desember mendatang. \n', '"Hari ini kami kembali melakukan uji coba KA Bandara dari Stasiun Sudirman Baru sampai dengan Stasiun Bandara Soekarno - Hatta. \n', 'Rencana operasional tanggal 1 - 3 Desember 2017 dengan 82 perjalanan setiap harinya dan\' headway\' setiap 30 menit," kata Menteri Perhubungan Budi Karya Sumadi dalam keterangan tertulis yang diterima di Bandung, Kamis (24 / 11 / 2017). \n', 'Sedangkan untuk tarifnya sendiri dibanderol Rp 100 ribu. \n', 'Namun khusus awal operasi Desember cuma Rp 30 rubu setelah diskon 70 persen. \n', '"Hal ini dilakukan sebagai upaya sosialisasi penggunaan transportasi publik yang baru kepada masyarakat, " katanya. \n', 'Menteri Perhubungan dalam uji coba ini didampingi oleh Zulfikri selaku Dirjen Perkeretaapian, Zamrides Direktur Prasarana DJKA, Budi Noviantoro selaku Direktur Logistik dan Pengembangan PT KAI Heru Kuswan

'Suara.com - Moda transportasi Kereta Api Bandara International Soekarno - Hatta di Cengkareng akan dioperasikan mulai awal Desember mendatang. \n"Hari ini kami kembali melakukan uji coba KA Bandara dari Stasiun Sudirman Baru sampai dengan Stasiun Bandara Soekarno - Hatta. \nRencana operasional tanggal 1 - 3 Desember 2017 dengan 82 perjalanan setiap harinya dan\' headway\' setiap 30 menit," kata Menteri Perhubungan Budi Karya Sumadi dalam keterangan tertulis yang diterima di Bandung, Kamis (24 / 11 / 2017). \nSedangkan untuk tarifnya sendiri dibanderol Rp 100 ribu. \nNamun khusus awal operasi Desember cuma Rp 30 rubu setelah diskon 70 persen. \n"Hal ini dilakukan sebagai upaya sosialisasi penggunaan transportasi publik yang baru kepada masyarakat, " katanya. \nMenteri Perhubungan dalam uji coba ini didampingi oleh Zulfikri selaku Dirjen Perkeretaapian, Zamrides Direktur Prasarana DJKA, Budi Noviantoro selaku Direktur Logistik dan Pengembangan PT KAI Heru Kuswanto selaku Direktur Utama 

In [ ]:

processed = article.replace(r'^\s+|\s+?$','')
processed = processed.replace('\n',' ')
processed = processed.replace("\\",'')
processed = processed.replace(",",'')
processed = processed.replace('"','')
processed = re.sub(r'\[[0-9]*\]','',processed)

processed

"Suara.com - Moda transportasi Kereta Api Bandara International Soekarno - Hatta di Cengkareng akan dioperasikan mulai awal Desember mendatang.  Hari ini kami kembali melakukan uji coba KA Bandara dari Stasiun Sudirman Baru sampai dengan Stasiun Bandara Soekarno - Hatta.  Rencana operasional tanggal 1 - 3 Desember 2017 dengan 82 perjalanan setiap harinya dan' headway' setiap 30 menit kata Menteri Perhubungan Budi Karya Sumadi dalam keterangan tertulis yang diterima di Bandung Kamis (24 / 11 / 2017).  Sedangkan untuk tarifnya sendiri dibanderol Rp 100 ribu.  Namun khusus awal operasi Desember cuma Rp 30 rubu setelah diskon 70 persen.  Hal ini dilakukan sebagai upaya sosialisasi penggunaan transportasi publik yang baru kepada masyarakat  katanya.  Menteri Perhubungan dalam uji coba ini didampingi oleh Zulfikri selaku Dirjen Perkeretaapian Zamrides Direktur Prasarana DJKA Budi Noviantoro selaku Direktur Logistik dan Pengembangan PT KAI Heru Kuswanto selaku Direktur Utama PT Railink Porwan

In [ ]:
sentences = sent_tokenize(processed)
sentences

['Suara.com - Moda transportasi Kereta Api Bandara International Soekarno - Hatta di Cengkareng akan dioperasikan mulai awal Desember mendatang.',
 'Hari ini kami kembali melakukan uji coba KA Bandara dari Stasiun Sudirman Baru sampai dengan Stasiun Bandara Soekarno - Hatta.',
 "Rencana operasional tanggal 1 - 3 Desember 2017 dengan 82 perjalanan setiap harinya dan' headway' setiap 30 menit kata Menteri Perhubungan Budi Karya Sumadi dalam keterangan tertulis yang diterima di Bandung Kamis (24 / 11 / 2017).",
 'Sedangkan untuk tarifnya sendiri dibanderol Rp 100 ribu.',
 'Namun khusus awal operasi Desember cuma Rp 30 rubu setelah diskon 70 persen.',
 'Hal ini dilakukan sebagai upaya sosialisasi penggunaan transportasi publik yang baru kepada masyarakat  katanya.',
 'Menteri Perhubungan dalam uji coba ini didampingi oleh Zulfikri selaku Dirjen Perkeretaapian Zamrides Direktur Prasarana DJKA Budi Noviantoro selaku Direktur Logistik dan Pengembangan PT KAI Heru Kuswanto selaku Direktur Utam

In [ ]:
frequency = {}
processed1 = processed.lower()
for word in word_tokenize(processed1):
    if word not in stop_word and word not in string.punctuation:
        if word not in frequency.keys():
            frequency[word]=1
        else:
            frequency[word]+=1
frequency

{'1': 3,
 '10': 1,
 '100': 1,
 '11': 1,
 '123': 1,
 '2017': 3,
 '24': 1,
 '272': 1,
 '3': 2,
 '30': 2,
 '33.728': 1,
 '50': 1,
 '70': 1,
 '82': 1,
 'ac': 1,
 'akan': 4,
 'allah': 1,
 'angkasa': 1,
 'antara': 2,
 'apbn': 1,
 'api': 2,
 'arahan': 1,
 'awal': 2,
 'bagi': 3,
 'bagian': 1,
 'bandara': 15,
 'bandung': 1,
 'baru': 3,
 'batuceper': 1,
 'berbasil': 1,
 'berbasis': 1,
 'bersama': 1,
 'bertransportasi': 1,
 'bertujuan': 1,
 'bisa': 3,
 'budi': 2,
 'cengkareng': 1,
 'cepat': 1,
 'coba': 3,
 'cuma': 1,
 'dalam': 3,
 'dan': 8,
 'dana': 1,
 'daop': 1,
 'dari': 6,
 'dengan': 7,
 'desember': 4,
 'di': 4,
 'dibanderol': 1,
 'didampingi': 1,
 'didanai': 1,
 'diharapkan': 2,
 'dilakukan': 2,
 'dioperasikan': 1,
 'direktur': 6,
 'dirjen': 1,
 'diskon': 1,
 'diterima': 1,
 'djka': 1,
 'duduk': 2,
 'electrical': 1,
 'emu': 1,
 'enam': 1,
 'fasilitas': 2,
 'full': 1,
 'hadir': 1,
 'hal': 1,
 'handry': 2,
 'hanya': 2,
 'hari': 2,
 'harinya': 1,
 'hatta': 5,
 'headway': 1,
 'helmy': 1,
 'hendi'

In [ ]:
max_fre = max(frequency.values())
print(frequency)
print(frequency['suara.com'])
print(max_fre)
for word in frequency.keys():
    frequency[word]=(frequency[word]/max_fre)
frequency

{'suara.com': 0.06666666666666667, 'moda': 0.06666666666666667, 'transportasi': 0.13333333333333333, 'kereta': 0.2, 'api': 0.13333333333333333, 'bandara': 1.0, 'international': 0.06666666666666667, 'soekarno': 0.3333333333333333, 'hatta': 0.3333333333333333, 'di': 0.26666666666666666, 'cengkareng': 0.06666666666666667, 'akan': 0.26666666666666666, 'dioperasikan': 0.06666666666666667, 'mulai': 0.13333333333333333, 'awal': 0.13333333333333333, 'desember': 0.26666666666666666, 'mendatang': 0.06666666666666667, 'hari': 0.13333333333333333, 'ini': 0.6, 'kami': 0.06666666666666667, 'kembali': 0.06666666666666667, 'melakukan': 0.06666666666666667, 'uji': 0.2, 'coba': 0.2, 'ka': 0.6, 'dari': 0.4, 'stasiun': 0.4, 'sudirman': 0.06666666666666667, 'baru': 0.2, 'sampai': 0.2, 'dengan': 0.4666666666666667, 'rencana': 0.06666666666666667, 'operasional': 0.06666666666666667, 'tanggal': 0.06666666666666667, '1': 0.2, '3': 0.13333333333333333, '2017': 0.2, '82': 0.06666666666666667, 'perjalanan': 0.066

{'1': 0.2,
 '10': 0.06666666666666667,
 '100': 0.06666666666666667,
 '11': 0.06666666666666667,
 '123': 0.06666666666666667,
 '2017': 0.2,
 '24': 0.06666666666666667,
 '272': 0.06666666666666667,
 '3': 0.13333333333333333,
 '30': 0.13333333333333333,
 '33.728': 0.06666666666666667,
 '50': 0.06666666666666667,
 '70': 0.06666666666666667,
 '82': 0.06666666666666667,
 'ac': 0.06666666666666667,
 'akan': 0.26666666666666666,
 'allah': 0.06666666666666667,
 'angkasa': 0.06666666666666667,
 'antara': 0.13333333333333333,
 'apbn': 0.06666666666666667,
 'api': 0.13333333333333333,
 'arahan': 0.06666666666666667,
 'awal': 0.13333333333333333,
 'bagi': 0.2,
 'bagian': 0.06666666666666667,
 'bandara': 1.0,
 'bandung': 0.06666666666666667,
 'baru': 0.2,
 'batuceper': 0.06666666666666667,
 'berbasil': 0.06666666666666667,
 'berbasis': 0.06666666666666667,
 'bersama': 0.06666666666666667,
 'bertransportasi': 0.06666666666666667,
 'bertujuan': 0.06666666666666667,
 'bisa': 0.2,
 'budi': 0.13333333333

In [ ]:
sentence_score = {}
for sent in sentences:
    for word in word_tokenize(sent):
        if word in frequency.keys():
            if len(sent.split(' '))<30:
                if sent not in sentence_score.keys():
                    sentence_score[sent] = frequency[word]
                else:
                    sentence_score[sent]+=frequency[word]

sentence_score

{'Dengan pengoperasian KA Bandara Soetta ini nanti diharapkan bisa menambah pilihan bagi masyarakat dalam mengakses bandara terbesar di Indonesia itu.': 3.3333333333333335,
 'Diharapkan KA Bandara ini bisa menarik bagi masyarakat dan mampu mengalihkan sebagian pemerjalan ke Bandara yang semula menggunakan kendaraan jalan raya ke KA Bandara ini.': 3.9333333333333336,
 'Hal ini dilakukan sebagai upaya sosialisasi penggunaan transportasi publik yang baru kepada masyarakat  katanya.': 2.2666666666666666,
 'Hari ini kami kembali melakukan uji coba KA Bandara dari Stasiun Sudirman Baru sampai dengan Stasiun Bandara Soekarno - Hatta.': 2.2666666666666666,
 'Jika selama ini hanya tersedia pilihan bertransportasi berbasis jalan raya maka sebentar lagi akan hadir layanan public transport berbasil jalan rel yang mengusung kenyamanan bagi pelanggan.': 3.333333333333335,
 'Kereta Api Bandara bisa mengangkut 33.728 penumpang tiap hari nya dengan fasilitas premium mulai dari tempat duduk yang nyaman 

In [ ]:

import heapq
summary = heapq.nlargest(2,sentence_score,key = sentence_score.get)
summary = ' '.join(summary)

final = "SUMMARY:- \n  " + summary + "TEXT:-    "+processed
textfinal = final.encode('ascii','ignore')
textfinal = str(textfinal) 
final

"SUMMARY:- \n  Diharapkan KA Bandara ini bisa menarik bagi masyarakat dan mampu mengalihkan sebagian pemerjalan ke Bandara yang semula menggunakan kendaraan jalan raya ke KA Bandara ini. Kereta Api Bandara bisa mengangkut 33.728 penumpang tiap hari nya dengan fasilitas premium mulai dari tempat duduk yang nyaman full AC wifi sampai dengan fasilitas toilet.TEXT:-    Suara.com - Moda transportasi Kereta Api Bandara International Soekarno - Hatta di Cengkareng akan dioperasikan mulai awal Desember mendatang.  Hari ini kami kembali melakukan uji coba KA Bandara dari Stasiun Sudirman Baru sampai dengan Stasiun Bandara Soekarno - Hatta.  Rencana operasional tanggal 1 - 3 Desember 2017 dengan 82 perjalanan setiap harinya dan' headway' setiap 30 menit kata Menteri Perhubungan Budi Karya Sumadi dalam keterangan tertulis yang diterima di Bandung Kamis (24 / 11 / 2017).  Sedangkan untuk tarifnya sendiri dibanderol Rp 100 ribu.  Namun khusus awal operasi Desember cuma Rp 30 rubu setelah diskon 70 

In [ ]:
# !pip install bert-extractive-summarizer
!pip install sentencepiece


     |████████████████████████████████| 1.2MB 5.4MB/s 


In [ ]:
get_corona_summary=open('/content/readme.txt','r').read()

In [ ]:
from summarizer import Summarizer
model = Summarizer()
result = model(get_corona_summary, num_sentences=4)
summary = "".join(result)
print(summary)

Suara.com - Moda transportasi Kereta Api Bandara International Soekarno - Hatta di Cengkareng akan dioperasikan mulai awal Desember mendatang. Rencana operasional tanggal 1 - 3 Desember 2017 dengan 82 perjalanan setiap harinya dan' headway' setiap 30 menit," kata Menteri Perhubungan Budi Karya Sumadi dalam keterangan tertulis yang diterima di Bandung, Kamis (24 / 11 / 2017). Namun khusus awal operasi Desember cuma Rp 30 rubu setelah diskon 70 persen. Dengan pengoperasian KA Bandara Soetta ini nanti diharapkan bisa menambah pilihan bagi masyarakat dalam mengakses bandara terbesar di Indonesia itu. Diharapkan KA Bandara ini bisa menarik bagi masyarakat dan mampu mengalihkan sebagian pemerjalan ke Bandara yang semula menggunakan kendaraan jalan raya ke KA Bandara ini.


In [ ]:
type(model)

summarizer.model_processors.Summarizer

Real Text

Suara.com - Moda transportasi Kereta Api Bandara International Soekarno - Hatta di Cengkareng akan dioperasikan mulai awal Desember mendatang.  Hari ini kami kembali melakukan uji coba KA Bandara dari Stasiun Sudirman Baru sampai dengan Stasiun Bandara Soekarno - Hatta.  Rencana operasional tanggal 1 - 3 Desember 2017 dengan 82 perjalanan setiap harinya dan' headway' setiap 30 menit kata Menteri Perhubungan Budi Karya Sumadi dalam keterangan tertulis yang diterima di Bandung Kamis (24 / 11 / 2017).  Sedangkan untuk tarifnya sendiri dibanderol Rp 100 ribu.  Namun khusus awal operasi Desember cuma Rp 30 rubu setelah diskon 70 persen.  Hal ini dilakukan sebagai upaya sosialisasi penggunaan transportasi publik yang baru kepada masyarakat  katanya.  Menteri Perhubungan dalam uji coba ini didampingi oleh Zulfikri selaku Dirjen Perkeretaapian Zamrides Direktur Prasarana DJKA Budi Noviantoro selaku Direktur Logistik dan Pengembangan PT KAI Heru Kuswanto selaku Direktur Utama PT Railink Porwanto Handry Nugroho selaku Direktur Operasi dan Teknik PT Railink dan Hendi Helmy selaku Ka Daop 1 Jakarta.  Uji coba operasi KA Bandara Soetta pada kesempatan kali ini lebih bertujuan untuk mencoba pertama kali track baru dari Stasiun Batuceper sampai Stasiun Bandara Soekarno Hatta sepanjang 123 KM sebagai bagian dari persiapan ` soft launching' operasi KA Bandara Soetta yang Insya Allah akan dilakukan antara 1 - 3 Desember 2017 sesuai arahan Pak Menhub  kata Direktur Utama PT Railink Heru Kuswanto.  Direktur Operasi dan Teknik PT Railink Porwanto Handry Nugroho mengatakan akan siapkan 10 rangkaian KRL (EMU / electrical multiple unit) untuk KA Bandara Soekarno - Hatta dan KRL telah siap operasi di mana setiap rangkaian memiliki enam kereta dengan keseluruhan jumlah tempat duduk 272 per rangkaan.  Kereta Api Bandara bisa mengangkut 33.728 penumpang tiap hari nya dengan fasilitas premium mulai dari tempat duduk yang nyaman full AC wifi sampai dengan fasilitas toilet.  Sedangkan waktu tempuhnya pun relatif cepat hanya 50 menit dengan rute Stasiun Manggarai - Stasiun Bandara Soekarno Hatta  katanya.  Dengan pengoperasian KA Bandara Soetta ini nanti diharapkan bisa menambah pilihan bagi masyarakat dalam mengakses bandara terbesar di Indonesia itu.  Jika selama ini hanya tersedia pilihan bertransportasi berbasis jalan raya maka sebentar lagi akan hadir layanan public transport berbasil jalan rel yang mengusung kenyamanan bagi pelanggan.  Menteri Perhubungan mengapresiasi keberadaan proyek KA Bandara Soetta proyek ini didanai dari investasi PT Railink bersama kedua induk perusahaan (PT KAI dan PT Angkasa Pura II) tanpa menggunakan dana dari APBN.  Diharapkan KA Bandara ini bisa menarik bagi masyarakat dan mampu mengalihkan sebagian pemerjalan ke Bandara yang semula menggunakan kendaraan jalan raya ke KA Bandara ini.

Human

Moda transportasi Kereta Api Bandara International Soekarno - Hatta di Cengkareng akan dioperasikan mulai awal Desember mendatang. 
Tarif kereta ini dibanderol Rp 100 ribu. 
Namun khusus awal operasi Desember, terdapat harga promo sebesar Rp 30 ribu, setelah diskon 70 %. 
Kereta Api Bandara bisa mengangkut 33.728 penumpang tiap hari dengan fasilitas premium, mulai dari tempat duduk yang nyaman, AC, wifi dan toilet. 

Golden Label

Suara.com - Moda transportasi Kereta Api Bandara International Soekarno - Hatta di Cengkareng akan dioperasikan mulai awal Desember mendatang. Sedangkan untuk tarifnya sendiri dibanderol Rp 100 ribu. Namun khusus awal operasi Desember cuma Rp 30 rubu setelah diskon 70 persen. Kereta Api Bandara bisa mengangkut 33.728 penumpang tiap hari nya dengan fasilitas premium, mulai dari tempat duduk yang nyaman, full AC, wifi, sampai dengan fasilitas toilet.

Manual approach

Diharapkan KA Bandara ini bisa menarik bagi masyarakat dan mampu mengalihkan sebagian pemerjalan ke Bandara yang semula menggunakan kendaraan jalan raya ke KA Bandara ini. Kereta Api Bandara bisa mengangkut 33.728 penumpang tiap hari nya dengan fasilitas premium mulai dari tempat duduk yang nyaman full AC wifi sampai dengan fasilitas toilet

BERT

Suara.com - Moda transportasi Kereta Api Bandara International Soekarno - Hatta di Cengkareng akan dioperasikan mulai awal Desember mendatang. Rencana operasional tanggal 1 - 3 Desember 2017 dengan 82 perjalanan setiap harinya dan' headway' setiap 30 menit," kata Menteri Perhubungan Budi Karya Sumadi dalam keterangan tertulis yang diterima di Bandung, Kamis (24 / 11 / 2017). Namun khusus awal operasi Desember cuma Rp 30 rubu setelah diskon 70 persen. Dengan pengoperasian KA Bandara Soetta ini nanti diharapkan bisa menambah pilihan bagi masyarakat dalam mengakses bandara terbesar di Indonesia itu. Diharapkan KA Bandara ini bisa menarik bagi masyarakat dan mampu mengalihkan sebagian pemerjalan ke Bandara yang semula menggunakan kendaraan jalan raya ke KA Bandara ini.

SBERT

Direktur Operasi dan Teknik PT Railink Porwanto Handry Nugroho mengatakan akan siapkan 10 rangkaian KRL (EMU / electrical multiple unit) untuk KA Bandara Soekarno - Hatta dan KRL telah siap operasi di mana setiap rangkaian memiliki enam kereta dengan keseluruhan jumlah tempat duduk 272 per rangkaan.
Dengan pengoperasian KA Bandara Soetta ini nanti diharapkan bisa menambah pilihan bagi masyarakat dalam mengakses bandara terbesar di Indonesia itu.
Diharapkan KA Bandara ini bisa menarik bagi masyarakat dan mampu mengalihkan sebagian pemerjalan ke Bandara yang semula menggunakan kendaraan jalan raya ke KA Bandara ini.
Menteri Perhubungan dalam uji coba ini didampingi oleh Zulfikri selaku Dirjen Perkeretaapian Zamrides Direktur Prasarana DJKA Budi Noviantoro selaku Direktur Logistik dan Pengembangan PT KAI Heru Kuswanto selaku Direktur Utama PT Railink Porwanto Handry Nugroho selaku Direktur Operasi dan Teknik PT Railink dan Hendi Helmy selaku Ka Daop 1 Jakarta.
Suara.com - Moda transportasi Kereta Api Bandara International Soekarno - Hatta di Cengkareng akan dioperasikan mulai awal Desember mendatang.

In [ ]:
# !pip install -U sentence-transformers
# !git clone https://github.com/UKPLab/sentence-transformers.git
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
"""
This example uses LexRank (https://www.aaai.org/Papers/JAIR/Vol22/JAIR-2214.pdf)
to create an extractive summarization of a long document.

The document is splitted into sentences using NLTK, then the sentence embeddings are computed. We
then compute the cosine-similarity across all possible sentence pairs.

We then use LexRank to find the most central sentences in the document, which form our summary.

Input document: First section from the English Wikipedia Section
Output summary:
Located at the southern tip of the U.S. state of New York, the city is the center of the New York metropolitan area, the largest metropolitan area in the world by urban landmass.
New York City (NYC), often called simply New York, is the most populous city in the United States.
Anchored by Wall Street in the Financial District of Lower Manhattan, New York City has been called both the world's leading financial center and the most financially powerful city in the world, and is home to the world's two largest stock exchanges by total market capitalization, the New York Stock Exchange and NASDAQ.
New York City has been described as the cultural, financial, and media capital of the world, significantly influencing commerce, entertainment, research, technology, education, politics, tourism, art, fashion, and sports.
If the New York metropolitan area were a sovereign state, it would have the eighth-largest economy in the world.
"""
import sys

import nltk
from sentence_transformers import SentenceTransformer, util
import numpy as np
sys.path.insert(1, '/content/sentence-transformers/examples/applications/text-summarization/')

from LexRank import degree_centrality_scores



model = SentenceTransformer('paraphrase-distilroberta-base-v1')

# Our input document we want to summarize
# As example, we take the first section from Wikipedia
document = """
Suara.com - Moda transportasi Kereta Api Bandara International Soekarno - Hatta di Cengkareng akan dioperasikan mulai awal Desember mendatang. Hari ini kami kembali melakukan uji coba KA Bandara dari Stasiun Sudirman Baru sampai dengan Stasiun Bandara Soekarno - Hatta. Rencana operasional tanggal 1 - 3 Desember 2017 dengan 82 perjalanan setiap harinya dan' headway' setiap 30 menit kata Menteri Perhubungan Budi Karya Sumadi dalam keterangan tertulis yang diterima di Bandung Kamis (24 / 11 / 2017). Sedangkan untuk tarifnya sendiri dibanderol Rp 100 ribu. Namun khusus awal operasi Desember cuma Rp 30 rubu setelah diskon 70 persen. Hal ini dilakukan sebagai upaya sosialisasi penggunaan transportasi publik yang baru kepada masyarakat katanya. Menteri Perhubungan dalam uji coba ini didampingi oleh Zulfikri selaku Dirjen Perkeretaapian Zamrides Direktur Prasarana DJKA Budi Noviantoro selaku Direktur Logistik dan Pengembangan PT KAI Heru Kuswanto selaku Direktur Utama PT Railink Porwanto Handry Nugroho selaku Direktur Operasi dan Teknik PT Railink dan Hendi Helmy selaku Ka Daop 1 Jakarta. Uji coba operasi KA Bandara Soetta pada kesempatan kali ini lebih bertujuan untuk mencoba pertama kali track baru dari Stasiun Batuceper sampai Stasiun Bandara Soekarno Hatta sepanjang 123 KM sebagai bagian dari persiapan ` soft launching' operasi KA Bandara Soetta yang Insya Allah akan dilakukan antara 1 - 3 Desember 2017 sesuai arahan Pak Menhub kata Direktur Utama PT Railink Heru Kuswanto. Direktur Operasi dan Teknik PT Railink Porwanto Handry Nugroho mengatakan akan siapkan 10 rangkaian KRL (EMU / electrical multiple unit) untuk KA Bandara Soekarno - Hatta dan KRL telah siap operasi di mana setiap rangkaian memiliki enam kereta dengan keseluruhan jumlah tempat duduk 272 per rangkaan. Kereta Api Bandara bisa mengangkut 33.728 penumpang tiap hari nya dengan fasilitas premium mulai dari tempat duduk yang nyaman full AC wifi sampai dengan fasilitas toilet. Sedangkan waktu tempuhnya pun relatif cepat hanya 50 menit dengan rute Stasiun Manggarai - Stasiun Bandara Soekarno Hatta katanya. Dengan pengoperasian KA Bandara Soetta ini nanti diharapkan bisa menambah pilihan bagi masyarakat dalam mengakses bandara terbesar di Indonesia itu. Jika selama ini hanya tersedia pilihan bertransportasi berbasis jalan raya maka sebentar lagi akan hadir layanan public transport berbasil jalan rel yang mengusung kenyamanan bagi pelanggan. Menteri Perhubungan mengapresiasi keberadaan proyek KA Bandara Soetta proyek ini didanai dari investasi PT Railink bersama kedua induk perusahaan (PT KAI dan PT Angkasa Pura II) tanpa menggunakan dana dari APBN. Diharapkan KA Bandara ini bisa menarik bagi masyarakat dan mampu mengalihkan sebagian pemerjalan ke Bandara yang semula menggunakan kendaraan jalan raya ke KA Bandara ini.
"""

#Split the document into sentences
sentences = nltk.sent_tokenize(document)
print("Num sentences:", len(sentences))

#Compute the sentence embeddings
embeddings = model.encode(sentences, convert_to_tensor=True)

#Compute the pair-wise cosine similarities
cos_scores = util.pytorch_cos_sim(embeddings, embeddings).numpy()

#Compute the centrality for each sentence
centrality_scores = degree_centrality_scores(cos_scores, threshold=None)

#We argsort so that the first element is the sentence with the highest score
most_central_sentence_indices = np.argsort(-centrality_scores)


#Print the 5 sentences with the highest scores
print("\n\nSummary:")
for idx in most_central_sentence_indices[0:5]:
    print(sentences[idx].strip())

Num sentences: 15


Summary:
Direktur Operasi dan Teknik PT Railink Porwanto Handry Nugroho mengatakan akan siapkan 10 rangkaian KRL (EMU / electrical multiple unit) untuk KA Bandara Soekarno - Hatta dan KRL telah siap operasi di mana setiap rangkaian memiliki enam kereta dengan keseluruhan jumlah tempat duduk 272 per rangkaan.
Dengan pengoperasian KA Bandara Soetta ini nanti diharapkan bisa menambah pilihan bagi masyarakat dalam mengakses bandara terbesar di Indonesia itu.
Diharapkan KA Bandara ini bisa menarik bagi masyarakat dan mampu mengalihkan sebagian pemerjalan ke Bandara yang semula menggunakan kendaraan jalan raya ke KA Bandara ini.
Menteri Perhubungan dalam uji coba ini didampingi oleh Zulfikri selaku Dirjen Perkeretaapian Zamrides Direktur Prasarana DJKA Budi Noviantoro selaku Direktur Logistik dan Pengembangan PT KAI Heru Kuswanto selaku Direktur Utama PT Railink Porwanto Handry Nugroho selaku Direktur Operasi dan Teknik PT Railink dan Hendi Helmy selaku Ka Daop 1 Jakarta.

In [ ]:
!git clone https://github.com/alexvlis/extractive-document-summarization.git

Cloning into 'extractive-document-summarization'...
remote: Enumerating objects: 7267, done.
remote: Total 7267 (delta 0), reused 0 (delta 0), pack-reused 7267
Receiving objects: 100% (7267/7267), 122.43 MiB | 30.19 MiB/s, done.
Resolving deltas: 100% (1090/1090), done.
Checking out files: 100% (3311/3311), done.


In [ ]:
!cd extractive-document-summarization/
!conda create --name <env> --file requirements.txt
!source activate <env> 
!pyrouge_set_rouge_path /global/pathto/extractive-document-summarization/preprocessing/pyrouge/tools/ROUGE-1.5.5/

!source deactivate

/bin/bash: env: No such file or directory
/bin/bash: -c: line 0: syntax error near unexpected token `newline'
/bin/bash: -c: line 0: `source activate <env> '
/bin/bash: pyrouge_set_rouge_path: command not found
/bin/bash: deactivate: No such file or directory


In [ ]:
!python /content/extractive-document-summarization/train.py

2021-05-10 08:39:55.279584: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
loading pickle files...
Traceback (most recent call last):
  File "/content/extractive-document-summarization/train.py", line 132, in <module>
    main()
  File "/content/extractive-document-summarization/train.py", line 107, in main
    x_train, y_train = load_data()
  File "/content/extractive-document-summarization/train.py", line 62, in load_data
    data1 = pickle.load(open("/global/scratch/alex_vlissidis/wordEmbeddingsToSaliency1.pickle", "rb"))
FileNotFoundError: [Errno 2] No such file or directory: '/global/scratch/alex_vlissidis/wordEmbeddingsToSaliency1.pickle'
